# Arcadia Ball Picking ROI Analysis
#### Arcadia Tractor
#### January 20, 2023

## Background

Golf driving ranges are labor intensive operations.  This notebook provides a high level economic analysis of the ROI of using an [Arcadia Tractor](www.arcadiatractor.com) to pick balls on a driving range.  The report sizes up the costs of operating a driving range in the state of **California** using off the shelf equipment and minimum wage labor.  This is then compared to the cost of picking balls using Arcadia's technology.  For reference, the report also examines the import of a $7.00 / hour minimum wage on the ROI analysis.

## Range Capacity

The total **range capacity** is a function of hours of operation and the hours of operation.  Typically, golfers strike one ball every 30 to 45 seconds seconds this results in ~100 balls per bay per hour.  Some driving ranges have lights and allow golfers to practice at night, while others don't. So the capacity is set by the following:

$$
C = hours \times bays \times 100
$$

## Actual Balls Hit / Day

The number of balls hit per day depends largely on the capacity utilization of the driving range.  

$$
T = C \times U
$$

where:

$T$ is the total balls hit per day  
$C$ is the capacity of the driving range  
$U$ is the range utilization (as a fraction)

### Sample Calculation

Let's evaluate the capacity for a driving range with:

- 12 hours of operation, and  
- 50 bays 


In [1]:
C = 12 * 50 * 100
C

60000

## Effort Required

A few factors contribute to the ball picking effort:

- Average time per trip  
- Storage capacity of the ball picker (gangs)  
- Speed of the tractor  
- Size/configuration of the Range

### Average Time Per Trip

Depending on the range, the average time per trip can vary.  However, this analysis assumes 25 minutes per trip as a starting point.

### Ball Picker Storage Capacity

A ball picker's capacity is a function of the number of baskets (usually called gangs) that can hold golf balls.  Smaller driving ranges with 20k / day or fewer balls typically have 3 gang ball pickers while larger capacity ranges will have 5 gangs.

### Optimum Driving Speed

There is a trade-off between driving speed and the thoroughness of ball picking.  Also, driving fast can lead to more equipment failures.  New operators tend to drive too fast and this often creates additional repair costs.

### Size and Range Configuration

Depending on the size of the range and where balls get deposited, the effort required to manage the range can vary. For example, if the ball washing occurs far away from where the balls get dispensed, more time is needed to transport balls to the client dispensing area. 


All of the above considerations can be captured by the following two numbers:
- time per trip (in minutes), and  
- balls per trip -- which is assumed to be 1,500 / trip for small capacity ranges and 2,500 per trip for high capacity ranges.


In [2]:
TIME_PER_TRIP = 25 # minutes

def get_hours_of_work(balls_per_day):
    if balls_per_day < 20000:
        balls_per_trip = 1500
    else:
        balls_per_trip = 2500
    pick_capacity_per_hour = balls_per_trip / ( TIME_PER_TRIP / 60)
    return balls_per_day / pick_capacity_per_hour

## Tractor & Hardware Cost

There are two schools of thought on how to equip a driving range.  Some range operators prefer having relatively new / low-cost equipment.  Others, prefer having older equipment that is fully depreciated from other parts of the golf course operation.  Most high capacity driving ranges also maintain a backup solution to avoid the risk of downtime for the range.

### Old / Semi Retired Equipment

In this model, the equipment cost is close to 0, but the annual maintenance and repair costs can be high.  Perhaps as much as 500 USD / month excluding the cost of downtime.

### New Tractor

Here, the operation might purchase a 15k USD tractor with an expected life of 3 years.  A new tractor will not cost very much in maintenance, but it is safe to budget 1,200 USD per year for maintenance.

### Other Costs

The ball picker costs anywhere from 2k to 5k USD and should be replaced every 3 years.  The cost of operating a diesel engine is 1 gallon per hour.

## Model Assumption

This model assumes the range operates with a new tractor with a 3 year life.  You can easily modify model to refelct different assumptions:

In [3]:
TRACTOR_COST = 15000 # dollars
TRACTOR_LIFE = 3 # years
TRACTOR_MAINTENANCE = 1200 # dollars / year
INSURANCE = 400 # dollars per year
FUEL = 4.00 # dollars per gallon

def get_tractor_cost_per_day(hours_worked):
    depreciation = (TRACTOR_COST / TRACTOR_LIFE) / 365
    other = (TRACTOR_MAINTENANCE + INSURANCE) / 365
    fuel = hours_worked * FUEL
    return depreciation + other + fuel

## Labor Costs

The cost of labor extends beyond the minimum wage.  The [California Minimum Wage](https://www.dds.ca.gov/rc/vendor-provider/minimum-wage/) is 15.00 USD per hour as of January 1, 2022.

### Federal Withholdings

[Federal withholdings](https://www.adp.com/resources/tools/tax-guides-and-forms/state-and-local-tax-guides/state-tax-guide.aspx) addup to 13% of wages.


### State Withholdings

In California, [state witholdings](https://edd.ca.gov/pdf_pub_ctr/de44.pdf) add up to 5% of wages.

### Workers Compensation / OSHA

These costs can be significant but they are ignored for the purpose of this analysis

In [4]:
WORKER_WAGE = 15 # dollars / hour
CA_WITHHOLDINGS = .05 # rate of wages
FEDERAL_WITHHOLDINGS = 0.062 + 0.145 + .06 # Social Security + Medicare + FUTA

def get_labor_cost(hours_worked, hourly_rate=15):
    worker = hourly_rate * hours_worked * (1 + CA_WITHHOLDINGS + FEDERAL_WITHHOLDINGS)
    return worker

### Management Effort

Managing the labor on the driving range is a significant effort.  The management duties range from hiring, firing, scheduling of shifts, training, and supervision.  This analysis assumes one day per week (8 hours / week) of effort to manage the crew of workers.

### Absenteism

If workers fail to show up at the range, the manager / supervisor is expected to pick balls.  This analysis assumes workers fail to show up 1 day per month.


In [5]:
MGMT_WAGE = 30 # dollars / hour
MGMT_EFFORT = 8 # hours per week to perform management duties
ABSENTEISM = 1 / 30 # propotion of time missed -- assumes manager picks the balls

def get_management_cost(hours_worked):
    daily_wages = MGMT_WAGE * MGMT_EFFORT / 7
    absent_wages = MGMT_EFFORT * (MGMT_WAGE - WORKER_WAGE) * hours_worked * ABSENTEISM
    total_cost = (daily_wages + absent_wages) * (1 + CA_WITHHOLDINGS + FEDERAL_WITHHOLDINGS)
    return total_cost

## Total Cost

This formula compiles the cost of running the driving range from the number of **balls hit per day**.  By working through the assumptions listed above, we compute the hours per day of effort and from there, it is possible to compute:

- the tractor cost,
- the cost of labor, and
- the cost of supervision

The model assumes 30 days per month of costs.

In [6]:
def get_manual_cost_per_month(balls_per_day, hourly_rate=15):
    hours_worked = get_hours_of_work(balls_per_day)
    tractor_cost = get_tractor_cost_per_day(hours_worked)
    labor_cost = get_labor_cost(hours_worked, hourly_rate)
    mgmt_cost = get_management_cost(hours_worked)
    return (tractor_cost + labor_cost + mgmt_cost) * 30

## Arcadia Cost

Arcadia offers two different tractors which differ in terms of battery capacity and motor horse power.  The light duty tractor can pick 30,000 balls per day and the heavy duty tractor can pick ~70,000 balls per day.  For applications that exceed 70,000 balls per day, it is recommended that customers purchase two heavy duty tractors.

In [7]:
def get_arcadia_cost_per_month(balls_per_day):
    if balls < 30000:
        a_cost = 2000 
    else:
        a_cost = (int(balls_per_day / 70000) + 1) * 3000
    return a_cost

In [9]:
import pandas as pd
rpt = []
for balls in range(10000,100000,10000):
    ca_manual_cost = get_manual_cost_per_month(balls, hourly_rate=15)
    usa_manual_cost = get_manual_cost_per_month(balls, hourly_rate=7)
    arcadia_cost = get_arcadia_cost_per_month(balls)
    r = {'balls': balls,
         'ca_manual_cost': ca_manual_cost,
         'usa_manual_cost': usa_manual_cost,
         'arcadia_cost': arcadia_cost}
    rpt += [r]
df = pd.DataFrame(rpt)

In [10]:
df

,balls,ca_manual_cost,usa_manual_cost,arcadia_cost
0,10000,4315.677658,3437.677658,2000
1,20000,4799.394325,3745.794325,2000
2,30000,6250.544325,4670.144325,3000
3,40000,7701.694325,5594.494325,3000
4,50000,9152.844325,6518.844325,3000
5,60000,10603.994325,7443.194325,3000
6,70000,12055.144325,8367.544325,6000
7,80000,13506.294325,9291.894325,6000
8,90000,14957.444325,10216.244325,6000


In [12]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['balls'], y=df['arcadia_cost'], name="Arcadia",
                    line_shape='hv'))

fig.add_trace(go.Scatter(x=df['balls'], y=df['ca_manual_cost'], name="California Minimum Wage Cost",
                    line_shape='linear'))
fig.add_trace(go.Scatter(x=df['balls'], y=df['usa_manual_cost'], name="USA Minimum Wage Cost",
                    line_shape='linear'))

fig.update_traces(hoverinfo='text+name', mode='lines+markers')
fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16))
fig.update_layout(title='Monthly Cost of Ball Picking<br>California',
                   xaxis_title='Balls / Day',
                   yaxis_title='USD')
fig.show()

### Arcadia Benefits
Arcadia's autonomous vehicle technology can help automate driving range operations and provide significant benefits to practice facility operators:

- **Eliminate Labor**: the ball picking job goes away, labor is still needed to wash the balls, but proper range design can make this a once per day task.
- **Eliminate Tractor Cost / Maintenance**: Arcadia provides the Robot as a Service (RaaS). This means that Arcadia provides the hardware and is responsibility for maintenance and service.
- **Little or No Down Time**: Arcadia provides a Service Level Agreement that provides a high level of uptime.  Arcadia's tractor runs off the cloud. By continuously monitoring the tractor, Arcadia can detect issues and dispatch a crew to provide field service when needed.

## Full Capture of Economic Benefit

In order to optimize the economic performance of the driving range, a few best practices should be followed.  Adopting an automated ball picking solution is the perfect time to rethink the workflow of the driving range operations.  Arcadia's **Applications Engineers** can sit with you to help design the most efficient workflow that will maximize the impact of automation on your operation.

## Book A Demo

To book a demo please get in touch with Arcadia Tractor by sending an email to: info@arcadiatractor.com.